In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.metrics import error_rate
import wandb
import os
from wandb.fastai import WandbCallback

In [3]:
os.environ['TORCH_HOME'] = '../torch-zoo'

In [6]:
wandb.init(project='aptos-2019', name='FastAI Resnet-152')

W&B Run: https://app.wandb.ai/basedrhys/aptos-2019/runs/okk7xo61

In [5]:
bs = 16

In [6]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

aptos_stats = ([0.42, 0.22, 0.075], [0.27, 0.15, 0.081])
data = ImageDataBunch.from_df(path='data/train_images/', 
                              df=train_df, 
                              suffix = '.png', 
                              ds_tfms=get_transforms(flip_vert=True), 
                              size=224, 
                              bs=bs).normalize(aptos_stats)

In [7]:
# show some sample images
data.show_batch(rows=3)

In [10]:
learn = cnn_learner(data, models.resnet152, metrics=[error_rate], callback_fns=WandbCallback)
learn.path = Path('models')

In [9]:
learn.fit_one_cycle(35)

epoch,train_loss,valid_loss,error_rate,time
0,1.275141,0.785593,0.254098,03:07
1,1.058115,0.807180,0.222678,03:44
2,0.983604,0.674444,0.206284,03:41
3,0.930576,0.728099,0.232240,03:33
4,0.841217,0.594200,0.215847,03:28
5,0.727693,0.550058,0.198087,03:35
6,0.664234,0.536621,0.184426,03:34
7,0.673355,0.494915,0.166667,03:32
8,0.614000,0.494026,0.166667,03:28
9,0.643332,0.465655,0.174863,03:34


Better model found at epoch 0 with valid_loss value: 0.7855931520462036.
Better model found at epoch 2 with valid_loss value: 0.6744440793991089.
Better model found at epoch 4 with valid_loss value: 0.5941997766494751.
Better model found at epoch 5 with valid_loss value: 0.5500581860542297.
Better model found at epoch 6 with valid_loss value: 0.5366210341453552.
Better model found at epoch 7 with valid_loss value: 0.49491456151008606.
Better model found at epoch 8 with valid_loss value: 0.49402615427970886.
Better model found at epoch 9 with valid_loss value: 0.4656551778316498.
Better model found at epoch 10 with valid_loss value: 0.45660609006881714.
Better model found at epoch 12 with valid_loss value: 0.42623233795166016.
Better model found at epoch 13 with valid_loss value: 0.4102703928947449.
Better model found at epoch 16 with valid_loss value: 0.41020143032073975.
Better model found at epoch 18 with valid_loss value: 0.40952858328819275.
Better model found at epoch 19 with vali

In [ ]:
learn.save('rn152_35')

# Running Model On Test Data

In [ ]:
tta_params = {'beta':0.12, 'scale':1.0}
sample_df = pd.read_csv('data/sample_submission.csv')
sample_df.head()

In [ ]:
learn.data.add_test(ImageList.from_df(
    sample_df,
    Path('data/test')
))

In [ ]:
preds, y = learn.TTA(ds_type=DatasetType.Test, **tta_params)

In [ ]:
os.makedirs('submissions', exist_ok=True)

In [ ]:
classes = ['Black-grass',
'Charlock',
'Cleavers',
'Common Chickweed',
'Common wheat',
'Fat Hen',
'Loose Silky-bent',
'Maize',
'Scentless Mayweed',
'Shepherds Purse',
'Small-flowered Cranesbill',
'Sugar beet']

In [ ]:
classes = [classes[i] for i in preds.argmax(1)]

In [ ]:
sample_df.species = classes

In [ ]:
sample_df.head()

In [ ]:
sample_df.to_csv('submissions/initial_submission.csv', index=False)
_ = sample_df.hist()